## UTILS

In [40]:
pip install python-sat[pblib,aiger]


In [41]:
import numpy as np
import pandas as pd

setseed = np.random.seed(15)

In [42]:
from pysat.examples.rc2 import RC2
from pysat.formula import WCNF, WCNFPlus
from pysat.card import *
from pysat.solvers import Minicard, Solver
solve = RC2(WCNF(), solver ='gc41')


In [43]:
# set number of parents and number of teachers
T = 4
P = 5
# random assigment parent/teacher
S = np.random.binomial(n=1, p=0.6, size=[T*P]).reshape((P,T))
print(S.shape)
print(S)

# get max number of meetings
meetings = [np.count_nonzero(S[:,c] == 1) for c in range(T)]
spots = max(meetings) 

#set a max number of meetings per teacher,
#spots= 6


def m(t, p, s):
    return 1 + p* T * spots + s * T + t 



(5, 4)
[[0 1 1 1]
 [1 1 1 1]
 [1 1 0 1]
 [0 0 0 1]
 [1 1 1 1]]


## Variable encodings

In [44]:
decoding_list =[]

for p in range(P):
  for s in range(spots):
        for t in range(T):
            print(f'{(t,p,s)} is encoded in {m(t,p,s)}')
            decoding_list.append((t,p,s))

(0, 0, 0) is encoded in 1
(1, 0, 0) is encoded in 2
(2, 0, 0) is encoded in 3
(3, 0, 0) is encoded in 4
(0, 0, 1) is encoded in 5
(1, 0, 1) is encoded in 6
(2, 0, 1) is encoded in 7
(3, 0, 1) is encoded in 8
(0, 0, 2) is encoded in 9
(1, 0, 2) is encoded in 10
(2, 0, 2) is encoded in 11
(3, 0, 2) is encoded in 12
(0, 0, 3) is encoded in 13
(1, 0, 3) is encoded in 14
(2, 0, 3) is encoded in 15
(3, 0, 3) is encoded in 16
(0, 0, 4) is encoded in 17
(1, 0, 4) is encoded in 18
(2, 0, 4) is encoded in 19
(3, 0, 4) is encoded in 20
(0, 1, 0) is encoded in 21
(1, 1, 0) is encoded in 22
(2, 1, 0) is encoded in 23
(3, 1, 0) is encoded in 24
(0, 1, 1) is encoded in 25
(1, 1, 1) is encoded in 26
(2, 1, 1) is encoded in 27
(3, 1, 1) is encoded in 28
(0, 1, 2) is encoded in 29
(1, 1, 2) is encoded in 30
(2, 1, 2) is encoded in 31
(3, 1, 2) is encoded in 32
(0, 1, 3) is encoded in 33
(1, 1, 3) is encoded in 34
(2, 1, 3) is encoded in 35
(3, 1, 3) is encoded in 36
(0, 1, 4) is encoded in 37
(1, 1, 4) 

In [45]:
# variables encoding, in soft form since i will use cardinality constraint w.r.t. S

for s in range(spots):
    for p in range(P):
        for t in range(T):
            solve.add_clause([m(t,p,s)],100)
            

## Other constraints


A teacher cannot meet 2 parents at the same time

A parent cannot meet 2 teachers at the same time

There is only one meeting per pair

- $ m_{t,p,s} \to \lnot m_{t,p',s} \forall p' \neq p  $
- $ m_{t,p,s} \to  \lnot m_{t',p,s} \forall t' \neq t  $
- $ m_{t,p,s} \to  \lnot m_{t,p,s'} \forall s' \neq s  $

In [46]:
# spot1 -> not spot2: CNF not spot1 or not spot2
for t in range(T):
    for p in range(P):
        for s in range(spots):
            for s_2 in range(s+1,spots):
                solve.add_clause([-m(t,p,s), -m(t,p,s_2)])


# a parent cannot meet 2 teachers at the same time

for s in range(spots):
    for p in range(P):
        for t in range(T):
            for t_2 in range(t+1,T):
                solve.add_clause([-m(t,p,s), -m(t_2,p,s)])
# teachers cannot meet 2 parents at the same time
for s in range(spots):
    for t in range(T):
        for p in range(P):
            for p_2 in range(p+1,P):
                solve.add_clause([-m(t,p,s), -m(t,p_2,s)])

In [47]:

w = WCNFPlus()
# at most one meeting for each 1 in S, and no meetings for 0 in S
# use WCNFPlus since it has atmost parameter

for t in range(T):
    for p in range(P):
        
        w.append([[m(t, p, s) for s in range(spots)],
                    int(S[p, t])], is_atmost=True)



In [48]:
temp = [solve.add_clause(clause) for clause in w.atms]


In [49]:
# First solution

model = solve.compute()

index=[c-1 for c in model if c>0]

array=np.array(decoding_list)



### Visualize schedules

In [50]:

Parents= ["Parent " + str(i) for i in range(P)]
Slot_number = ["Slot " + str(i) for i in range(spots)]
Teachers = ["Teacher " + str(i) for i in range(T)]

Tables = []

for c in range(spots):
    Table = [""] * T
    Tables.append(Table)
    for t, p, s in array[index,] :
        if s == c:
            Table[t] = Parents[p]
  


pd.DataFrame(Tables, index = Slot_number, columns = Teachers)

df1=pd.DataFrame(Tables, index = Slot_number, columns = Teachers)
print(f'The following are set to true:{[c for c in model if c>0]}')
print(df1)

The following are set to true:[4, 6, 11, 21, 28, 34, 39, 42, 45, 52, 80, 87, 90, 96, 97]
       Teacher 0 Teacher 1 Teacher 2 Teacher 3
Slot 0  Parent 1  Parent 2            Parent 0
Slot 1  Parent 2  Parent 0  Parent 4  Parent 1
Slot 2            Parent 4  Parent 0  Parent 2
Slot 3            Parent 1            Parent 4
Slot 4  Parent 4            Parent 1  Parent 3


## ENCODE PREFERENCE

We prefer schedules where teachers do not stay idle, i.e. solutions with consecutive time slots. This che be encoded in the following way:

- if solt i is occupied -> also next solt is occupied
- $\forall t,p \quad m_{t,p,s} \to m_{t,p,s+1}$ in CNF $ \lnot {m_{t,p,s}} \lor m_{t,p,s+1} $

For these clauses a weight is assigend.

In [51]:

for t in range(T):
  for p in range(P):
      for s in range(spots):
          for s_2 in range(s+1,spots):
                solve.add_clause([-m(t,p,s), m(t,p,s_2)], weight=10)



In [52]:
model = solve.compute()

index=[c-1 for c in model if c>0]
print(f'The following are set to true:{[c for c in model if c>0]}')

array=np.array(decoding_list)


Parents = ["Parent " + str(i) for i in range(P)]
Slot_number = ["Slot " + str(i) for i in range(spots)]
Teachers = ["Teacher " + str(i) for i in range(T)]

Tables = []

for c in range(spots):
    Table = [""] * T
    Tables.append(Table)
    for t, p, s in array[index,] :
        if s == c:
            Table[t] = Parents[p]
  


df2=pd.DataFrame(Tables, index = Slot_number, columns = Teachers)
print(df2)

The following are set to true:[4, 11, 14, 28, 30, 35, 37, 49, 56, 58, 80, 86, 92, 93, 99]
       Teacher 0 Teacher 1 Teacher 2 Teacher 3
Slot 0                                Parent 0
Slot 1            Parent 4            Parent 1
Slot 2  Parent 2  Parent 1  Parent 0  Parent 4
Slot 3  Parent 4  Parent 0  Parent 1  Parent 2
Slot 4  Parent 1  Parent 2  Parent 4  Parent 3
